In [1]:
!pip install pyjsgf
!pip install rdflib

In [2]:
from jsgf import parse_grammar_string, filter_expansion

import rdflib

Knowledge

In [3]:
URI_PREFIX = 'http://kgdemo.com/'

triples = [
  ('宝马', 'is_what', '宝马（BMW）是德国豪华汽车品牌'),
  ('宝马', 'is_how', '德系大品牌值得信赖，各方面口碑都很好'),
  ('宝马', 'is_compared', '各有千秋，但是人生苦短，我选宝马'),
  ('捷豹', 'is_what', '捷豹（Jaguar）英国豪华汽车品牌，英国皇室御用品牌，1935年诞生'),
  ('宾利', 'is_how', '举世闻名的豪华汽车制造品牌，非常昂贵哦'),
  ('帕加尼', 'is_what', '帕加尼（Pagani）是一家位于意大利摩德纳的超级跑车制造商，该车厂坚持手工打造车辆，其汽车产量非常少，价格也十分昂贵'),
  ('广汽本田', 'is_what', '广汽本田汽车有限公司（原广州本田汽车有限公司；简称广汽本田）于1998年7月1日成立，它是由广州汽车集团公司与日本本田技研工业株式会社共同出资组建的合资公司，双方各占50%股份，合作年限为30年'),
  ('北京奔驰', 'is_how', '大品牌值得信赖，我经常在宝马的后视镜里看到它'),
]

graph = rdflib.Graph()
resources = set([r for triple in triples for r in triple])
resource2uri = {r: URI_PREFIX + r for r in resources}
uri2resource = {uri: r for r, uri in resource2uri.items()}
for (s, p, o) in triples:
  s_uri = rdflib.URIRef(resource2uri[s])
  p_uri = rdflib.URIRef(resource2uri[p])
  o_uri = rdflib.URIRef(resource2uri[o])
  graph.add((s_uri, p_uri, o_uri))

Rule

In [4]:
grammar = parse_grammar_string(
  "#JSGF V1.0 UTF-8 en ;"
  "grammar example ;"
  "public <top> = <is_what>{car} | <is_how>{car} | <is_compared>{car};"
  "<is_what> = (<cb_t> [是] 什么) | (<introduce> <cb_t> [这个汽车品牌]);"
  "<is_how> = (<cb_t> [这个牌子的] [汽车] (怎么样|如何[呢]|好用吗));"
  "<is_compared> = (<cb_t> 和 <cb> 比 (怎么样|哪个[更]好));"
  "<cb_t> = (宝马{t})|(捷豹{t})|(宾利{t})|(帕加尼{t})|(广汽本田{t})|(北京奔驰{t})|(奔驰{t});"
  "<cb> = (宝马)|(捷豹)|(宾利)|(帕加尼)|(广汽本田)|(北京奔驰)|(奔驰);"
  "<introduce> = (我想了解[一下]) | (给我(介绍|讲解)一下);"
)

In [5]:
def retrieve_kg(graph, db_query, uri2resource):
  print(db_query)
  for row in graph.query(db_query):
    print('Output:', uri2resource[row.o.toPython()])
    print()
    print()


for utt in ['宝马是什么',
            '我想了解一下宝马',
            '给我介绍一下宝马',
            '给我讲解一下捷豹这个汽车品牌',
            '给我介绍一下帕加尼',
            '我想了解一下广汽本田',
            '宝马这个牌子的汽车怎么样',
            '宾利这个牌子的汽车怎么样',
            '北京奔驰怎么样',
            '宝马如何呢',
            '宝马汽车好用吗',
            '宝马和奔驰比怎么样',
            '宝马和奔驰比哪个好',
            '宝马和奔驰比哪个更好',]:
  print('Query:', utt)
  rules = grammar.find_matching_rules(utt)
  if len(rules) > 0:
    rule = rules[0]
    tagged_expansions = filter_expansion(
      rule.expansion,
      lambda e: e.tag and e.had_match,)
    
    intent, entity = [], []
    for res in set(tagged_expansions):
      if hasattr(res, 'text'):
        entity.append(res.text)
      if hasattr(res, 'name'):
        intent.append(res.name)

    if len(intent) > 0 and len(entity) > 0:
      intent = intent[0]
      entity = entity[0]

      if intent == 'is_what':
        db_query = (
        """
        PREFIX : <%s>

        SELECT DISTINCT ?o WHERE {
            :%s :is_what ?o .
        }
        """ % (URI_PREFIX, entity)
        )
        retrieve_kg(graph, db_query, uri2resource)

      if intent == 'is_how':
        db_query = (
        """
        PREFIX : <%s>
        
        SELECT DISTINCT ?o WHERE {
            :%s :is_how ?o .
        }
        """ % (URI_PREFIX, entity)
        )
        retrieve_kg(graph, db_query, uri2resource)
      
      if intent == 'is_compared':
        db_query = (
        """
        PREFIX : <%s>
        
        SELECT DISTINCT ?o WHERE {
            :%s :is_compared ?o .
        }
        """ % (URI_PREFIX, entity)
        )
        retrieve_kg(graph, db_query, uri2resource)

Query: 宝马是什么

        PREFIX : <http://kgdemo.com/>

        SELECT DISTINCT ?o WHERE {
            :宝马 :is_what ?o .
        }
        
Output: 宝马（BMW）是德国豪华汽车品牌


Query: 我想了解一下宝马

        PREFIX : <http://kgdemo.com/>

        SELECT DISTINCT ?o WHERE {
            :宝马 :is_what ?o .
        }
        
Output: 宝马（BMW）是德国豪华汽车品牌


Query: 给我介绍一下宝马

        PREFIX : <http://kgdemo.com/>

        SELECT DISTINCT ?o WHERE {
            :宝马 :is_what ?o .
        }
        
Output: 宝马（BMW）是德国豪华汽车品牌


Query: 给我讲解一下捷豹这个汽车品牌

        PREFIX : <http://kgdemo.com/>

        SELECT DISTINCT ?o WHERE {
            :捷豹 :is_what ?o .
        }
        
Output: 捷豹（Jaguar）英国豪华汽车品牌，英国皇室御用品牌，1935年诞生


Query: 给我介绍一下帕加尼

        PREFIX : <http://kgdemo.com/>

        SELECT DISTINCT ?o WHERE {
            :帕加尼 :is_what ?o .
        }
        
Output: 帕加尼（Pagani）是一家位于意大利摩德纳的超级跑车制造商，该车厂坚持手工打造车辆，其汽车产量非常少，价格也十分昂贵


Query: 我想了解一下广汽本田

        PREFIX : <http://kgdemo.com/>

        SELECT DISTINCT ?o WHERE {
     